<a href="https://colab.research.google.com/github/Rioto3/portfolio-kpi/blob/feature%2Fanalytics-dashboard/analysis/portfolio_analytics_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 初期設定



In [1]:
# 必要なライブラリ
!pip install pandas
!pip install plotly
from google.cloud import bigquery
import pandas as pd
import plotly.express as px


In [2]:
# BigQueryへの接続設定
from google.colab import auth
import pandas as pd
auth.authenticate_user()

# プロジェクトIDの設定
project_id = 'digital-portfolio-analytics-bq'

### (例) 2025/02/21 データ

In [10]:

# 1件のデータを取得するクエリ
sample_query = """
SELECT *
FROM `digital-portfolio-analytics-bq.analytics_479207480.events_20250221`
LIMIT 1
"""

# クエリを実行してデータフレームに格納
sample_data = pd.read_gbq(sample_query, project_id=project_id)

# データの確認
print("データ構造:")
print(sample_data.shape)
print("\nカラム一覧:")
print(sample_data.columns.tolist())
print("\nサンプルデータ:")
sample_data

<ipython-input-10-dd8402063f58>:11: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  sample_data = pd.read_gbq(sample_query, project_id=project_id)


データ構造:
(1, 30)

カラム一覧:
['event_date', 'event_timestamp', 'event_name', 'event_params', 'event_previous_timestamp', 'event_value_in_usd', 'event_bundle_sequence_id', 'event_server_timestamp_offset', 'user_id', 'user_pseudo_id', 'privacy_info', 'user_properties', 'user_first_touch_timestamp', 'user_ltv', 'device', 'geo', 'app_info', 'traffic_source', 'stream_id', 'platform', 'event_dimensions', 'ecommerce', 'items', 'collected_traffic_source', 'is_active_user', 'batch_event_index', 'batch_page_id', 'batch_ordering_id', 'session_traffic_source_last_click', 'publisher']

サンプルデータ:


,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,event_dimensions,ecommerce,items,collected_traffic_source,is_active_user,batch_event_index,batch_page_id,batch_ordering_id,session_traffic_source_last_click,publisher
0,20250221,1740101034247317,first_visit,"[{'key': 'medium', 'value': {'string_value': '...",<NA>,NaN,-670759787,<NA>,None,1708590628.1740101033,...,None,None,[],"{'manual_campaign_id': None, 'manual_campaign_...",True,1,1740101033081,1,"{'manual_campaign': {'campaign_id': None, 'cam...",None


### (メイン)

In [33]:
# 1. イベントデータのサンプル確認
sample10_query = """
SELECT
  event_date,
  event_name,
  user_pseudo_id,
  (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'engagement_time_msec') as engagement_time_msec,
  (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_location') as page_location
FROM `digital-portfolio-analytics-bq.analytics_479207480.events_20250222`
LIMIT 10
"""

sample10 = pd.read_gbq(sample10_query, project_id=project_id)
sample10



<ipython-input-33-a7e8c5c68224>:13: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  sample10 = pd.read_gbq(sample10_query, project_id=project_id)


,event_date,event_name,user_pseudo_id,engagement_time_msec,page_location
0,20250222,first_visit,1333354421.1740191660,<NA>,http://localhost/
1,20250222,session_start,1333354421.1740191660,<NA>,http://localhost/
2,20250222,page_view,1333354421.1740191660,<NA>,http://localhost/
3,20250222,user_engagement,1333354421.1740191660,3640,http://localhost/
4,20250222,session_start,1333354421.1740191660,<NA>,http://localhost/
5,20250222,page_view,1333354421.1740191660,<NA>,http://localhost/
6,20250222,user_engagement,1333354421.1740191660,3303,http://localhost/


In [43]:
# 日付確認クエリ
date_check_query = """
SELECT
  _TABLE_SUFFIX as table_date,
  COUNT(*) as event_count
FROM `digital-portfolio-analytics-bq.analytics_479207480.events_*`
GROUP BY _TABLE_SUFFIX
ORDER BY _TABLE_SUFFIX DESC
"""

date_check = pd.read_gbq(date_check_query, project_id=project_id)
date_check

<ipython-input-43-75b3fcdca1da>:11: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  date_check = pd.read_gbq(date_check_query, project_id=project_id)


,table_date,event_count
0,20250222,7
1,20250221,4


In [46]:

kpi_analysis_query = """
SELECT
  PARSE_DATE('%Y%m%d', event_date) as date,
  COUNT(DISTINCT user_pseudo_id) as unique_visitors,
  COUNTIF(event_name = 'page_view') as pageviews,
  AVG(IF(event_name = 'user_engagement',
      (SELECT value.int_value FROM UNNEST(event_params) WHERE key = 'engagement_time_msec'),
      NULL)) as avg_engagement_ms,
  COUNT(DISTINCT (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_location')) as unique_pages
FROM `digital-portfolio-analytics-bq.analytics_479207480.events_*`
WHERE _TABLE_SUFFIX BETWEEN '20250220' AND '20250222'
GROUP BY date
ORDER BY date DESC
"""

kpi_results = pd.read_gbq(kpi_analysis_query, project_id=project_id)
kpi_results


<ipython-input-46-0100004fda8d>:16: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  kpi_results = pd.read_gbq(kpi_analysis_query, project_id=project_id)


,date,unique_visitors,pageviews,avg_engagement_ms,unique_pages
0,2025-02-22,1,2,3471.5,1
1,2025-02-21,1,2,NaN,1
